In [2]:
import sklearn
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import *
from scipy.linalg import sqrtm
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.manifold import *
from sklearn.preprocessing import *
from sklearn.metrics import *
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn.neighbors import KNeighborsClassifier
import pickle
from catboost import *
import gc
import catboost
from sklearn.base import clone
from collections import *

In [3]:
data = pd.read_parquet("../input/train-all-fill/train_repeat.parquet")

In [4]:
# types = dict(data.dtypes)

# for feature in types.keys():
#     if types[feature] == np.int32:
#         data[feature] = data[feature].astype(np.int16)
#     elif types[feature] == np.float32:
#         data[feature] = data[feature].astype(np.float16)

data = data.astype(np.float16)

In [5]:
labels = pd.read_csv("../input/amex-default-prediction/train_labels.csv")
labels = labels.set_index("customer_ID").astype(np.int16)

In [6]:
# labels_repeat = pd.read_parquet("../input/train-all-fill/labels_repeat.parquet")
# labels_repeat = labels_repeat.astype(np.int16)

In [7]:
# #array([0.79046377, 0.79385093, 0.79316176, 0.79531271])
# #array([0.79155378, 0.79284569, 0.7913045 , 0.79387503])
# x = data.groupby(["customer_ID"]).mean()

# model = catboost.CatBoostClassifier(early_stopping_rounds=10, iterations=300)

# cross_val_score(model, x.to_numpy(np.float16), labels["target"], cv = 4, scoring="f1")

In [8]:
cat_features  = ['B_30', 'B_38', 'D_114', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64',  'D_68']

In [9]:
data = data.drop(cat_features, axis=1)

In [10]:
data_group = data.groupby(data.index)

data_max  = data_group.max()
data_min  = data_group.min()
data_mean = data_group.mean()

data_stat_features = pd.concat([data_max, data_min, data_mean], axis = 1).to_numpy(np.float16)

# del data_max
# del data_min
# del data_mean

In [11]:
data = data.to_numpy(np.float16)
labels = labels.to_numpy(np.int16)

In [12]:
data = data.reshape(-1, 13, data.shape[1])
labels = labels.reshape(-1)

In [13]:
train_data, test_data, train_label, test_label = train_test_split(data, labels, test_size = 0.3, stratify = labels, random_state = 0, shuffle = True)
train_stat_features, test_stat_features = data_stat_features[:train_data.shape[0]], data_stat_features[train_data.shape[0]:]

In [14]:
kfolds = StratifiedKFold(3)

In [ ]:
# def create_train_row_models(data, labels, model):
        
#     models = []
        
#     for i in tqdm(range(data.shape[1])):
        
#         train_data = data[:, i, :]
        
#         model_ = model
        
#         model_.fit(train_data, labels)
        
#         models.append(model_)
    
#     return models

# def create_train_column_models(data, labels, model):
    
#     models = []
    
#     for i in tqdm(range(data.shape[2])):
        
#         train_data = data[:, :, i]
        
#         model_ = model

#         model_.fit(train_data, labels)
        
#         models.append(model_)
        
#     return models

# def predict_by_column_models(data, models):
    
#     results = np.zeros((data.shape[0], data.shape[2]))
    
#     for i in tqdm(range(data.shape[2])):
        
#         test_data = data[:, :, i]
        
#         preds = models[i].predict(test_data)
        
#         results[:, i] = preds
        
#     return results


# def predict_by_row_models(data, models):
        
#     results = np.zeros((data.shape[0], data.shape[1]))

#     for i in tqdm(range(data.shape[1])):
        
#         test_data = data[:, i, :]

#         preds = models[i].predict(test_data)
        
#         results[:, i] = preds
    
#     return results


# def predict_first_level_models(data, row_models, column_models):
    
#     row_preds    = predict_by_row_models(data, row_models)
#     column_preds = predict_by_column_models(data, column_models)

#     main_preds = np.concatenate((row_preds, column_preds), axis = 1)
    
#     return main_preds

In [16]:
class Model():
    
    def __init__(self, main_model, row_model, column_model,
                 count_row_models = 13, count_column_models = 156,
                 main_model_path = None, row_model_path = None, column_model_path = None):
        
        self.main_model    = main_model
        self.row_model     = row_model
        self.column_model  = column_model
        
        self.count_row_models    = count_row_models
        self.count_column_models = count_column_models 

        self.main_model_path     = main_model_path
        self.row_model_path      = row_model_path
        self.column_model_path   = column_model_path
        
        self.init_first_level_models()
        
        if self.main_model_path != None:
            self.main_model  = pickle.load(open(self.main_model_path, "rb"))
        else:
            self.main_model = self.main_model.copy()
            

    def init_first_level_models(self):
        
        if self.row_model_path != None:
            self.row_models = pickle.load(open(self.row_model_path, "rb"))
        else:
            self.row_models = [self.row_model.copy() for i in range(self.count_row_models)]
        
        if self.column_model_path != None:
            self.column_models = pickle.load(open(self.column_model_path, "rb"))
        else:
            self.column_models = [clone(self.column_model) for i in range(self.count_column_models)]
    
    
    def train_row_models(self, data, labels):

        for i in tqdm(range(data.shape[1])):

            train_data = data[:, i, :]

            self.row_models[i].fit(train_data, labels)

        
    def train_column_models(self, data, labels):

        for i in tqdm(range(data.shape[2])):

            train_data = data[:, :, i]

            self.column_models[i].fit(train_data, labels)

    
    def train_main_model(self, data, labels, additional_features = None):
            
        main_preds = self.predict_first_level_models(data)

        if type(additional_features) != type(None):
            main_preds = np.concatenate([main_preds, additional_features], axis = 1)

        self.main_model.fit(main_preds, labels)
        
        return main_preds
    
    
    def predict(self, data, additional_features = None):
        
        main_preds = self.predict_first_level_models(data)
        
        if type(additional_features) != type(None):
            main_preds = np.concatenate([main_preds, additional_features], axis= 1)
        
        preds = self.main_model.predict(main_preds)
        
        return preds
    
    
    def predict_by_column_models(self, data):

        results = np.zeros((data.shape[0], data.shape[2]))

        for i in tqdm(range(data.shape[2])):

            test_data = data[:, :, i]

            preds = self.column_models[i].predict(test_data)

            results[:, i] = preds

        return results

    
    def predict_by_row_models(self, data):

        results = np.zeros((data.shape[0], data.shape[1]))

        for i in tqdm(range(data.shape[1])):

            test_data = data[:, i, :]

            preds = self.row_models[i].predict(test_data)

            results[:, i] = preds

        return results


    def predict_first_level_models(self, data):

        row_preds    = self.predict_by_row_models(data)
        column_preds = self.predict_by_column_models(data)
        
        main_preds = np.concatenate([row_preds, column_preds], axis = 1)

        return main_preds

In [17]:
# main_model   = CatBoostClassifier(depth = 9, iterations = 600, l2_leaf_reg = 3,  learning_rate = 0.01, task_type="CPU", logging_level = "Silent")
# row_model    = CatBoostClassifier(early_stopping_rounds=10, task_type="CPU", logging_level = "Silent") 
# column_model = LogisticRegression(max_iter = 500)

# model = Model(main_model,
#               row_model, 
#               column_model, 
#               13, 156,
#              "./main_model.pkl",
#              "./row_models.pkl",
#              "./column_models.pkl")

EOFError: Ran out of input

In [18]:
main_model   = CatBoostClassifier(depth = 9, iterations = 600, l2_leaf_reg = 3,  learning_rate = 0.01, task_type="CPU", logging_level = "Silent")
row_model    = CatBoostClassifier(early_stopping_rounds=10, task_type="CPU", logging_level = "Silent") 
column_model = LogisticRegression(max_iter = 500)

model = Model(main_model,
              row_model, 
              column_model)

In [ ]:
for train1_index, train2_index in kfolds.split(train_data, train_label):

    train1, train1_targets = train_data[train1_index], train_label[train1_index]
    train2,  train2_targets  = train_data[train2_index],  train_label[train2_index]
    train_stat   = train_stat_features [train2_index]
    
    model.train_row_models(   train1, train1_targets)
    model.train_column_models(train1, train1_targets)
        
    model.train_main_model(train2, train2_targets)

    model.init_first_level_models()

In [ ]:
model.train_column_models(train_data, train_label)
model.train_row_models(train_data, train_label)

In [4]:
with open("./model_simple3.pkl", "rb") as file:
    model = pickle.load(file)

In [20]:
preds = model.predict(test_data)

100%|██████████| 156/156 [00:09<00:00, 16.59it/s]


In [17]:
for train_index, test_index in kfolds.split(train_data, train_label):

    train, train_targets = train_data[train_index], train_label[train_index]
    test,  test_targets  = train_data[test_index],  train_label[test_index]
    
    
    column_models = create_train_column_models(train, train_targets, column_model)
    row_models    = create_train_row_models(   train, train_targets, row_model)
    
    
    main_train = predict_first_level_models(data, row_models, column_models)
    
    main_model.fit(main_train, test_targets)

100%|██████████| 156/156 [00:07<00:00, 20.95it/s]


Learning rate set to 0.075791
0:	learn: 0.6159359	total: 23ms	remaining: 23s
1:	learn: 0.5514880	total: 45.3ms	remaining: 22.6s
2:	learn: 0.4983928	total: 68.6ms	remaining: 22.8s
3:	learn: 0.4563492	total: 90.3ms	remaining: 22.5s
4:	learn: 0.4222724	total: 113ms	remaining: 22.4s
5:	learn: 0.3962103	total: 135ms	remaining: 22.4s
6:	learn: 0.3751100	total: 161ms	remaining: 22.9s
7:	learn: 0.3585264	total: 188ms	remaining: 23.3s
8:	learn: 0.3444210	total: 213ms	remaining: 23.4s
9:	learn: 0.3321184	total: 236ms	remaining: 23.3s
10:	learn: 0.3217767	total: 259ms	remaining: 23.3s
11:	learn: 0.3124045	total: 282ms	remaining: 23.2s
12:	learn: 0.3047967	total: 305ms	remaining: 23.1s
13:	learn: 0.2988255	total: 327ms	remaining: 23s
14:	learn: 0.2939004	total: 350ms	remaining: 23s
15:	learn: 0.2884948	total: 374ms	remaining: 23s
16:	learn: 0.2856760	total: 396ms	remaining: 22.9s
17:	learn: 0.2818119	total: 421ms	remaining: 23s
18:	learn: 0.2794690	total: 445ms	remaining: 23s
19:	learn: 0.2769084	

100%|██████████| 156/156 [00:08<00:00, 18.63it/s]


Learning rate set to 0.075791
0:	learn: 0.6139374	total: 23.6ms	remaining: 23.6s
1:	learn: 0.5494713	total: 47.2ms	remaining: 23.6s
2:	learn: 0.5008592	total: 72.9ms	remaining: 24.2s
3:	learn: 0.4564078	total: 99.4ms	remaining: 24.7s
4:	learn: 0.4242721	total: 123ms	remaining: 24.5s
5:	learn: 0.3979828	total: 148ms	remaining: 24.5s
6:	learn: 0.3757647	total: 175ms	remaining: 24.8s
7:	learn: 0.3572879	total: 199ms	remaining: 24.6s
8:	learn: 0.3429850	total: 222ms	remaining: 24.5s
9:	learn: 0.3320873	total: 245ms	remaining: 24.3s
10:	learn: 0.3224217	total: 271ms	remaining: 24.3s
11:	learn: 0.3139541	total: 296ms	remaining: 24.4s
12:	learn: 0.3068320	total: 322ms	remaining: 24.5s
13:	learn: 0.3010883	total: 345ms	remaining: 24.3s
14:	learn: 0.2951170	total: 368ms	remaining: 24.2s
15:	learn: 0.2912161	total: 397ms	remaining: 24.4s
16:	learn: 0.2882129	total: 420ms	remaining: 24.3s
17:	learn: 0.2847420	total: 443ms	remaining: 24.2s
18:	learn: 0.2819693	total: 466ms	remaining: 24.1s
19:	lea

100%|██████████| 156/156 [00:07<00:00, 19.57it/s]


Learning rate set to 0.075791
0:	learn: 0.6192208	total: 23.4ms	remaining: 23.4s
1:	learn: 0.5506910	total: 47.3ms	remaining: 23.6s
2:	learn: 0.4987954	total: 73.4ms	remaining: 24.4s
3:	learn: 0.4581481	total: 96.3ms	remaining: 24s
4:	learn: 0.4268040	total: 120ms	remaining: 23.8s
5:	learn: 0.4023238	total: 144ms	remaining: 23.8s
6:	learn: 0.3800571	total: 166ms	remaining: 23.5s
7:	learn: 0.3629803	total: 191ms	remaining: 23.7s
8:	learn: 0.3469062	total: 215ms	remaining: 23.7s
9:	learn: 0.3353183	total: 239ms	remaining: 23.7s
10:	learn: 0.3246771	total: 263ms	remaining: 23.6s
11:	learn: 0.3167834	total: 287ms	remaining: 23.6s
12:	learn: 0.3101572	total: 312ms	remaining: 23.7s
13:	learn: 0.3043909	total: 336ms	remaining: 23.6s
14:	learn: 0.2988020	total: 358ms	remaining: 23.5s
15:	learn: 0.2950890	total: 384ms	remaining: 23.6s
16:	learn: 0.2912405	total: 409ms	remaining: 23.7s
17:	learn: 0.2877887	total: 432ms	remaining: 23.5s
18:	learn: 0.2846297	total: 454ms	remaining: 23.4s
19:	learn

In [66]:
first = True

for i in tqdm(range(1, 4)):
    data = pd.read_parquet("../input/amex-test-repeat/test{}.parquet".format(i)).astype(np.float16)
    
    cat_features  = ['B_30', 'B_38', 'D_114', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64',  'D_68']
    
    indexes = data[[]].drop_duplicates().index
    data = data.drop(cat_features, axis=1)
    
    #data_stat_features = get_stats(data)
    
    data = data.to_numpy(np.float16)
    data = data.reshape(-1, 13, data.shape[1])
    
    preds = predict(model, data)
    
    preds = pd.DataFrame({"customer_ID" : indexes, "prediction" : preds}, index = "customer_ID")
    
    if first:
        preds.to_csv("submit.csv")
        first = False
    else:
        preds.to_csv("submit.csv", mode = "a", header = False)

8519

In [23]:
with open("model_simple3.pkl", "wb") as file:
    pickle.dump(data[:10000], file)

In [20]:
with open("column_models.pkl", "wb") as file:
    pickle.dump(column_models, file)
with open("row_models.pkl", "wb") as file:
    pickle.dump(row_models, file)
with open("main_model.pkl", "wb") as file:
    pickle.dump(main_model, file)

In [38]:
main_model_grid = CatBoostClassifier(logging_level = "Silent")

parameters = {
                'depth'        : [3, 6, 9],
                'learning_rate': [0.01, 0.03, 0.07],
                'iterations'   : [600, 900],
                'l2_leaf_reg'  : [3, 5, 9],
                
             }

grid = GridSearchCV(estimator=main_model_grid, param_grid = parameters, cv = 2)
grid.fit(main_train, test_targets)

GridSearchCV(cv=2,
             estimator=<catboost.core.CatBoostClassifier object at 0x7ff1845573d0>,
             param_grid={'depth': [3, 6, 9], 'iterations': [600, 900],
                         'l2_leaf_reg': [3, 5, 9],
                         'learning_rate': [0.01, 0.03, 0.07]})

In [39]:
grid.best_params_

{'depth': 9, 'iterations': 600, 'l2_leaf_reg': 3, 'learning_rate': 0.01}

In [21]:
row_preds    = predict_by_row_models(test_data, row_models)
column_preds = predict_by_column_models(test_data, column_models)

main_test = np.concatenate((row_preds, column_preds), axis = 1)

100%|██████████| 156/156 [00:10<00:00, 15.30it/s]


In [32]:
#preds_def = main_model_def.predict(test_data[:, 12, :])
f1_score(test_label, preds)

0.7898702155741313

In [21]:
f1_score(test_label, preds)

0.7921135126216796

In [25]:
test1 = predict_by_models(test_data, models)

100%|██████████| 13/13 [01:34<00:00,  7.29s/it]


In [22]:
preds = main_model.predict(main_test)

In [57]:
accuracy_score(test_label, preds)

0.8900736522509697

In [28]:
f1_score(test_label, preds)

0.7775934986317273

In [ ]:
models = (data, )